In [21]:
from sklearn.model_selection import cross_val_score, cross_validate,cross_val_predict, StratifiedShuffleSplit
from datautils import*
import os
import os.path as path
from profiling import *
from learningutils import *
from sklearn import tree
import json
from matching_task import *
import datetime

sep_for_source_files= ','
gs_sep = ','
main_path='../datasets/'
source_folder ='magellan_datasets'


for r, d, f in os.walk(main_path+source_folder+'/'):
    print("Current path :"+r)
    
    if r!=main_path+source_folder+'/':
        dataset_name = r.split("/")[-1]

    else: continue
    ds1=pd.DataFrame()
    ds2=pd.DataFrame()
    gs=pd.DataFrame()
    for file in f:

        feature_vector_train,feature_vector_test,feature_vector_validation = None, None,  None

        if '.csv' in file and 'standard_2' in file:
            gs = pd.read_csv(r+'/'+file, sep=gs_sep)
        if path.exists(r+"/feature_vector_train.csv"):
            feature_vector_train = pd.read_csv(r+"/feature_vector_train.csv", sep =",", engine='python')
        if path.exists(r+"/feature_vector_test.csv"):
            feature_vector_test = pd.read_csv(r+"/feature_vector_test.csv", sep =",", engine='python')
        if path.exists(r+"/feature_vector_validation.csv"):
            feature_vector_validation = pd.read_csv(r+"/feature_vector_validation.csv", sep =",", engine='python')
        if '.csv' in file and 'standard' not in file and 'feature' not in file:
            if '1_' in file : ds1= pd.read_csv(r+'/'+file, sep =sep_for_source_files, engine='python')
            if '2_' in file:  ds2= pd.read_csv(r+'/'+file, sep =sep_for_source_files, engine='python')

        if not ds1.empty and not ds2.empty and not gs.empty: break    


    if not ds1.empty and not ds2.empty and not gs.empty and feature_vector_train is None and feature_vector_test is None and feature_vector_validation is None:
        ds1['subject_id'] = ds1['subject_id'].apply(str)
        ds2['subject_id'] = ds2['subject_id'].apply(str)

        if (source_folder=="leipzig_datasets" or source_folder=="corleone_datasets"):
            ds1['subject_id'] = ds1['subject_id'].str.lower()
            ds2['subject_id'] = ds2['subject_id'].str.lower()


        gs['source_id'] = gs['source_id'].apply(str)
        gs['target_id'] = gs['target_id'].apply(str)

        if path.exists(r+"/feature_vector.csv"):
            feature_vector = pd.read_csv(r+"/feature_vector.csv", sep =",", engine='python')
        else: 
            feature_vector = createFeatureVectorFile(ds1,ds2,gs, embeddings = False, printProgress=True, saveFile=r+"/feature_vector.csv")

        X = feature_vector
        y = feature_vector['label'].values
        sssplits_train_test = StratifiedShuffleSplit(n_splits=1, test_size=0.3, random_state=1)
        train_index, val_test_index = next(sssplits_train_test.split(X,y))
        X_train, X_val_test, y_train, y_val_test = X.loc[train_index], X.loc[val_test_index],y[train_index],y[val_test_index]
        X_val_test.reset_index(inplace=True)
        X_train.reset_index(inplace=True)   
        sssplits_val_test = StratifiedShuffleSplit(n_splits=1, test_size=0.33, random_state=1)
        val_index, test_index = next(sssplits_val_test.split(X_val_test, y_val_test))
        X_val, X_test, y_val, y_test = X_val_test.loc[val_index], X_val_test.loc[test_index], y_val_test[val_index], y_val_test[test_index]
        X_val.drop(columns=['index'], inplace=True)
        X_test.drop(columns=['index'], inplace=True)
        X_train.drop(columns=['index'], inplace=True)
        X_val.to_csv(r+"/feature_vector_validation.csv", index=False)
        X_test.to_csv(r+"/feature_vector_test.csv", index=False)
        X_train.to_csv(r+"/feature_vector_train.csv", index=False)

Current path :/work/aprimpel/datasets_for_notebooks/datasetProfilingtrain_test_val/magellan_datasets/bikes
